## Data Science Academy

### Projeto e Implementação de Plataforma de Dados com Snowflake

### Lab 9

### Implementando Uma Sala de Limpeza de Dados (Data Clean Room) Para Machine Learning

### Instalando e Carregando Pacotes

Instale o snowflake-ml-python conforme mostrado nas aulas.

In [ ]:
# Imports
import streamlit as st
import pandas as pd
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import when, col
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error
from snowflake.ml.modeling.metrics import r2_score
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import OneHotEncoder
from snowflake.ml.registry import Registry
import warnings
warnings.filterwarnings('ignore')

### Definindo a Sessão Snowpark

In [ ]:
# Captura a sessão Snowpark
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# Função para capturar informações da sessão
def dsa_current_snowflake_env():
    snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
    print('User                     : {}'.format(snowflake_environment[0][0]))
    print('Role                     : {}'.format(snowflake_environment[0][1]))
    print('Database                 : {}'.format(snowflake_environment[0][2]))
    print('Schema                   : {}'.format(snowflake_environment[0][3]))
    print('Warehouse                : {}'.format(snowflake_environment[0][5]))
    print('Snowflake version        : {}'.format(snowflake_environment[0][4]))

In [ ]:
# Executa a função
dsa_current_snowflake_env()

### Carregando os Dados

Crie database e schema e então crie uma tabela a partir do arquivo CSV.

In [ ]:
# Importa os dados da tabela como um dataframe
df_dsa_dados = session.table('DSA_DATABASE.DSA_SCHEMA.DSA_TABLE')

### Sala de Limpeza de Dados

In [ ]:
# Vamos gerar 2 novas colunas
df_dsa_dados = (
    df_dsa_dados
    .with_column('IDADE', 2025 - df_dsa_dados['ANO'])
    .with_column('NUM_PORTAS', df_dsa_dados["PORTAS"].substring(1,2))
)

In [ ]:
df_dsa_dados

In [ ]:
# Aplicamos filtros para limpeza inicial de dados (conforme regras de negócio definidas)
df_dsa_dados_clean = (
    df_dsa_dados
    .filter(F.col("KILOMETRAGEM") <= 500000)
    .filter(F.col("IDADE") <= 20 )
    .filter(F.col("TRANSMISSAO").in_(F.lit("Manual"), F.lit("Automatico")) )
    .filter(F.col("PRECO_VENDA") <= 100000)
    .filter(F.col("COMBUSTIVEL").in_(F.lit("Gasolina"), F.lit("Diesel")) )
)

In [ ]:
# Drop das colunas que não serão usadas
df_dsa_dados_clean = df_dsa_dados_clean.drop(["PRECO","POWER"] )

In [ ]:
# Resumo estatístico dos dados
df_dsa_dados_clean.describe()

In [ ]:
# Resumo estatístico dos dados (comando alternativo)
df_dsa_dados_clean.describe().show()

In [ ]:
dsa_top_modelos = (
    df_dsa_dados_clean
    .group_by("MODELO")
    .agg(F.count("MODELO").alias('N'))
    .filter(F.col('N') > 750)
    .withColumnRenamed('MODELO', 'MODELO2')
)

In [ ]:
dsa_top_modelos

In [ ]:
df_dsa_dados_clean_modelos = (
    df_dsa_dados_clean
    .join(dsa_top_modelos, df_dsa_dados_clean["MODELO"] == dsa_top_modelos["MODELO2"], "inner")
    .select(df_dsa_dados_clean["*"])
)

In [ ]:
df_dsa_dados_clean_modelos.count()

In [ ]:
df_dsa_dados_clean_modelos.show()

In [ ]:
# Lista de valores incorretos para cor do veículo
valores_incorretos = ["2-portas", "3-portas", "4-portas", "5-portas", "6-portas"]

# Substitui o valor da lista por None
df_dsa_dados_clean_modelos = df_dsa_dados_clean_modelos.withColumn(
    "COR",
    when(col("COR").isin(valores_incorretos), None).otherwise(col("COR"))
)

In [ ]:
# NA pode ser uma string mesmo e não apenas ausência de dado, e por isso pode ser necessário remover de forma explícita.
df_dsa_dados_clean_modelos = df_dsa_dados_clean_modelos.filter(col("COR") != "NA")

In [ ]:
# Drop NA 
df_dsa_dados_clean_modelos = df_dsa_dados_clean_modelos.dropna()

In [ ]:
# Lista de colunas categóricas
colunas_categoricas = ["MODELO", "COMBUSTIVEL", "TRANSMISSAO", "COR"]

In [ ]:
# Lista de colunas categóricas para aplicar One Hot Encoder
colunas_categoricas_ohe = ["MODELO_OE" ,"COMBUSTIVEL_OE", "TRANSMISSAO_OE", "COR_OE"]

In [ ]:
# Colunas numéricas
colunas_numericas = ['IDADE', 'KILOMETRAGEM']

In [ ]:
# Variável alvo e coluna para receber a previsão do modelo
coluna_target = ['PRECO_VENDA']
coluna_previsao = ['PRECO_PREVISTO']

In [ ]:
# Prepara o dataframe para treinar o modelo
df_dsa_dados_limpos_ml = (df_dsa_dados_clean_modelos.select(colunas_categoricas + colunas_numericas + coluna_target))

In [ ]:
df_dsa_dados_limpos_ml

In [ ]:
# Divisão dos dados em treino e teste
df_treino, df_teste = df_dsa_dados_limpos_ml.random_split(weights = [0.80, 0.20], seed = 0) 

### Criação do Modelo

In [ ]:
# Criação do modelo
modelo_dsa = RandomForestRegressor(n_estimators = 250, 
                                   max_depth = 6,
                                   n_jobs = -1,
                                   min_samples_split = 500,
                                   min_samples_leaf = 500,
                                   label_cols = coluna_target,
                                   output_cols = coluna_previsao)

### Pipeline com Machine Learning e Resultado da Sala de Limpeza

In [ ]:
# Criação do pipeline
dsa_ml_pipe = Pipeline(
    steps = [
        (
            "ohe", 
            OneHotEncoder(
                input_cols = colunas_categoricas, 
                output_cols = colunas_categoricas_ohe, 
                drop_input_cols = True
            )
        ),
        (
            "modelo_dsa",
            modelo_dsa
        )
    ]
)

In [ ]:
dsa_ml_pipe

### Treinamento do Modelo

In [ ]:
# Treina o modelo e salva o output
df_treino_out = dsa_ml_pipe.fit_predict(df_treino)

In [ ]:
# Visualiza
df_treino_out.show()

### Avaliação do Modelo

In [ ]:
# Previsões com dados de teste
df_teste_out = dsa_ml_pipe.predict(df_teste)

In [ ]:
# Calcula a métrica MAPE
mape = mean_absolute_percentage_error(df = df_teste_out, 
                                      y_true_col_names = "PRECO_VENDA", 
                                      y_pred_col_names = "PRECO_PREVISTO")

In [ ]:
print(mape)

In [ ]:
# Calcula a métrica R2 Score
r2 = r2_score(df = df_teste_out, 
              y_true_col_name = "PRECO_VENDA", 
              y_pred_col_name = "PRECO_PREVISTO")

In [ ]:
print(r2)

### App Para Testar o Modelo

In [ ]:
# Registra o modelo
reg = Registry(session = session, database_name = "DSA_DATABASE", schema_name = "DSA_SCHEMA")

In [ ]:
reg

In [ ]:
# Registra a versão do modelo
modelo_versao = reg.log_model(dsa_ml_pipe,
                              model_name = "modelo_dsa_deploy",
                              version_name = "v1",
                              comment = "Modelo DSA")

In [ ]:
modelo_versao

In [ ]:
# Previsões com o modelo
previsoes = modelo_versao.run(df_dsa_dados_limpos_ml, function_name = "predict")

In [ ]:
# Visualiza
previsoes.show()

In [ ]:
# Salva a tabela com as previsões
df_dsa_dados_limpos_ml.write.mode("overwrite").save_as_table("DSA_DATABASE.DSA_SCHEMA.DSA_PREVISOES")

In [ ]:
SELECT * FROM DSA_DATABASE.DSA_SCHEMA.DSA_PREVISOES

### App Para Usar o Pipeline de Machine Learning

In [ ]:
# Extrai a lista de modelos
dsa_top_modelos_lista = dsa_top_modelos.to_pandas().MODELO2.values

In [ ]:
# Cria a app com Streamlit

# Define as colunas
c1,c2,c3 = st.columns(3)
with c2:
    IDADE = st.number_input("IDADE",0,20,2)
    COMBUSTIVEL = st.selectbox("COMBUSTIVEL", [ "Gasolina", "Diesel"])
with c3:
    KILOMETRAGEM = st.number_input("KILOMETRAGEM",15000)
    TRANSMISSAO = st.selectbox("TRANSMISSAO",["Manual", "Automatico"])
with c1:
    MODELO = st.selectbox("MODELO", dsa_top_modelos_lista)
    COR = st.selectbox("COR", [ "Preto", "Branco"])

# Divider
st.divider()

# Prepara o dataframe com dados de um novo carro
novo_carro = pd.DataFrame({
    "MODELO": [MODELO],
    "IDADE" : [IDADE],
    "KILOMETRAGEM": [KILOMETRAGEM],
    "TRANSMISSAO" :[TRANSMISSAO],
    "COMBUSTIVEL" : [COMBUSTIVEL],
    "COR" : [COR]
})

# Converte o dataframe do pandas em dataframe do Snowpark
novo_carro_df = session.create_dataframe(novo_carro)

# Faz previsões com o ml_pipeline e seleciona a coluna de previsão
dsa_ml_pipe.predict(novo_carro_df).select("PRECO_PREVISTO")

# Fim